In [ ]:
import numpy as np
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import os
import sys
sys.path.append('/Users/admin/Desktop/tesi/Thesis/')
import cv2
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC
import sklearn.metrics as skmetrics
from sklearn.model_selection import GridSearchCV
from handcrafted_descriptors.grid_classification import utils
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support
species = ['Arthonia_radiata','Caloplaca_cerina','Candelariella_reflexa','Candelariella_xanthostigma','Chrysothrix_candelaris','Flavoparmelia_caperata','Gyalolechia_flavorubescens','Hyperphyscia_adglutinata'
        ,'Lecanora_argentata','Lecanora_chlarotera','Lecidella_elaeochroma','Melanelixia_glabratula'
        ,'Phaeophyscia_orbicularis','Physcia_biziana','Physconia_grisea','Ramalina_farinacea','Ramalina_fastigiata','Xanthomendoza_fallax','Xanthomendoza_fulva','flavoparmenia_soredians']

def load_descriptor_from_matfile(ft_path,lab_path):
    feat = loadmat(ft_path)
    feat = feat['dsc']
    lab = loadmat(lab_path)
    lab = lab['lab']
    return feat, lab



def gabor_kernel(u,v,vec):

    m = u.shape[0]
    res = 0
    for i in range(0,m,2):
        mu_u = u[i]
        mu_v = v[i]

        sig_u = u[i+1]
        sig_v = v[i+1]

        mu_sum = np.abs((mu_u - mu_v)/vec[i])
        sig_sum = np.abs((sig_u - sig_v)/vec[i+1])

        res += (mu_sum + sig_sum)
    res = np.exp(-res)
    return res



def gabor_distance(u,v,vec):
    m = u.shape[0]
    res = 0
    for i in range(0,m,2):
        mu_u = u[i]
        mu_v = v[i]

        sig_u = u[i+1]
        sig_v = v[i+1]

        mu_sum = np.abs((mu_u - mu_v)/vec[i])
        sig_sum = np.abs((sig_u - sig_v)/vec[i+1])

        res += (mu_sum + sig_sum)
    return res




def compute_gram_matrix(data,kernel):
    samples,_ = np.shape(data)
    print(samples)
    vec= np.std(data,axis = 0)
    matrix = np.zeros((samples,samples))
    for r in range(samples):
        print(r)
        for c in range(r,samples):
            tmp = kernel(data[r],data[c],vec)
            matrix[r,c] = tmp
            matrix[c,r] = tmp
    return matrix


def define_and_train_svm(tr_feature, tr_lab, kernel_type, distance = None):
    if distance == None:
        print('here')
        svc  = OneVsRestClassifier(SVC(kernel = kernel_type,gamma = 'scale'),n_jobs = -1)
        svc = svc.fit(tr_feature, tr_lab)
        return svc, 0, np.zeros(tr_feature.shape[0])
    else:
        gram=  compute_gram_matrix(tr_feature,distance)
        mean = np.mean(gram[np.triu_indices(np.shape(tr_feature)[0])])
        gram = np.exp(-(gram)) # generalized Gaussian kernel
        svc  = OneVsRestClassifier(SVC(kernel = 'precomputed'),n_jobs = -1)
        svc = svc.fit(gram,np.array(tr_lab))
        return svc, mean,gram

def test_accuracy(test_features,features,kernel,vec, test_label,svc):
    num_objects = features.shape[0]
    print(num_objects)
    print(test_features.shape[0])
    res = []
    prediction = []
    for i,ft in enumerate(test_features):
        print(i)

        pred = svc.predict(np.array([kernel(ft, features[num, :], vec) for num in range(num_objects)]).reshape(1, -1))
        prediction.append(pred[0])
        #print(i,": ",pred," : ",test_label[i])
        if(pred==test_label[i]):
            res.append(1)
        else:
            res.append(0)

    res = np.array(res).reshape(-1)
    return np.sum(res)/res.shape, np.array(prediction)



def train_model_and_calculate_accuracy(train_features,train_lab,test_features,test_lab): 
    scaler = StandardScaler()
    scaler.fit(train_features)

    X_train = scaler.transform(train_features)
    X_test = scaler.transform(test_features)
    clc = []
    acc = []
    prec = []
    rec = []
    for i in range(1,51):
        classifier = KNeighborsClassifier(n_neighbors=i) 
        classifier.fit(X_train, train_lab)
        clc.append(classifier)
        y_pred = classifier.predict(X_test)
        acc_temp = 0
        total = y_pred.shape[0]
        for j in range(y_pred.shape[0]):
            if y_pred[j]==test_lab[j]:
                acc_temp = acc_temp + 1
        precision, recall, fbeta, support = precision_recall_fscore_support(test_lab, y_pred,zero_division=0)
        prec.append(np.mean(precision))
        rec.append(np.mean(recall))
        acc.append(acc_temp/total)
    return acc, prec, rec,clc 



In [ ]:
import pandas as pd
def evaluated_prediction(pred,test_lab,lab ):
    """
    Function which evaluate quality of prdiction of linear svm, calculating TP,FP,FN,TN

    Input pred = prediction
    Input test_lab = labels
    Input num_classes(15) = number of classes
    Input lab = name of classes

    Output  res = dataframe with all these values
    """
    num_classes = len(lab)
    tp = []
    fp = []
    fn = []
    tn = []
    for i in range(num_classes):
        tp_temp = 0
        fp_temp = 0
        fn_temp = 0
        tn_temp = 0
        for j in range(len(pred)):
            if(pred[j]==lab[i] and test_lab[j]==lab[i]):
                tp_temp = tp_temp + 1
            if(pred[j]==lab[i] and test_lab[j]!=lab[i]):
                fp_temp = fp_temp + 1
            if(pred[j]!=lab[i] and test_lab[j]==lab[i]):
                fn_temp = fn_temp + 1
            if(pred[j]!=lab[i] and test_lab[j]!=lab[i]):

                tn_temp = tn_temp +1
        tp.append(tp_temp)
        fp.append(fp_temp)
        fn.append(fn_temp)
        tn.append(tn_temp)
    data = {'labels':lab , 'True positive':tp,'True negative':tn,'False positive':fp,'False negative':fn}
    res = pd.DataFrame(data, columns = ['labels','True positive','True negative','False positive','False negative'])
    return res


def build_confusion_matrix(df,pred,test_labels,lab):
    """
    Function tu construct confusion matrix
    """
    num_classes = len(lab)
    cm = np.zeros((num_classes,num_classes))
    # insert true positive on the diagonal
    for i in range(num_classes):
        cm[i,i] = df.loc[i]['True positive']
    for i in range(num_classes): # lavoro sulle classes true
        for j in range(num_classes): #lavoro su classes predicted
            temp = 0
            for k in range(len(test_labels)):
                if(test_labels[k]==lab[i] and pred[k]==lab[j]):
                    temp = temp +1
            cm[i,j]=temp
    return cm


In [ ]:
def search_for_best_rbf(training_feat, tr_lab, tst_features, tst_lab):
    param_grid = {'C': [ 1e0, 1e1, 1e2,1e3], 'gamma': [   0.01, 0.1,1]}
    clf0 = GridSearchCV(SVC(kernel='rbf'), param_grid)
    svc = clf0.fit(training_feat, tr_lab)
    print("Best estimator found by grid search : ", clf0.best_estimator_)
    y_pred = clf0.predict(tst_features)
    print('Accuracy score :', skmetrics.accuracy_score(y_pred, tst_lab))
    return clf0,skmetrics.accuracy_score(y_pred, tst_lab)

def search_for_best_poly(training_feat, tr_lab, tst_features, tst_lab):
    param_grid = {'degree':[2,3,4,5],'C': [ 1e0, 1e1, 1e2,1e3], 'gamma': [ 0.001, 0.01, 0.1,1]}
    clf0 = GridSearchCV(SVC(kernel='poly'), param_grid)
    svc = clf0.fit(training_feat, tr_lab)
    print("Best estimator found by grid search : ", clf0.best_estimator_)
    y_pred = clf0.predict(tst_features)
    print('Accuracy score :', skmetrics.accuracy_score(y_pred, tst_lab))
    return clf0, skmetrics.accuracy_score(y_pred, tst_lab)

def create_and_save_confusion_matrix(model, tst_features, tst_lab, species, vw,director, name ):
    pred = model.predict(tst_features)
    df = utils.evaluated_prediction(pred, tst_lab, species)
    cm = utils.build_confusion_matrix(df, pred, tst_lab,species)
    fig=plt.figure(figsize=(30, 15))
    utils.plot_confusion_matrix(cm,species,name,director,normalize=True,title='Confusion matrix')
    plt.close()
    return pred

import matplotlib.pyplot as plt
def plot_and_save_knn(acc,prec,rec,direct):
    x1 = np.arange(1,51)
    plt.plot(x1, acc, label = "accuracy",color = 'r')
    plt.plot(x1,prec, label = 'precision')
    plt.plot(x1, rec, label = 'recall',color = 'g')
    plt.xlabel('number of neighbors')
    plt.ylabel('percentage (%)')
    plt.legend()
    plt.grid()
    plt.savefig(direct)
    plt.close()

In [ ]:
vector = ['1','2','3','4','5','6','7','8']


acc_radial = {}
#acc_poly = {}
acc_1nn = {}

for i,rot in enumerate(vector):
    for j,scale in enumerate(vector):
        print('------------------')
        print('(' + rot + ',' + scale + ')')
        print('-------------------')
        string_training = 'dsc/' + str(rot) + '_' + str(scale) + '_' + 'train_descriptors.mat'
        string_training_label = 'dsc/' + str(rot) + '_' + str(scale) + '_' + 'train_labels.mat'       
        string_testing = 'dsc/' + str(rot) + '_' + str(scale) + '_' + 'test_descriptors.mat'
        string_testing_label = 'dsc/' + str(rot) + '_' + str(scale) + '_' + 'test_labels.mat'
        
        training_feat,tr_lab = load_descriptor_from_matfile(string_training, string_training_label)
        tst_features, tst_lab = load_descriptor_from_matfile(string_testing,string_testing_label)
        tr_lab = tr_lab -1
        tst_lab = tst_lab -1 
        #print('start rbf')
        #svm ,score_rbf = search_for_best_rbf(training_feat, tr_lab.reshape(-1), tst_features, tst_lab)
        #print('start poly')
        #svm ,score_poly = search_for_best_poly(training_feat, tr_lab.reshape(-1), tst_features, tst_lab)
        print('start 1nn')
        acc, prec, rec,clc  = train_model_and_calculate_accuracy(training_feat, tr_lab.reshape(-1), tst_features, tst_lab.reshape(-1))
        #acc_radial['(' + str(rot) + ',' + str(scale) + ')'] = score_rbf
        #acc_poly['(' + str(rot) + ',' + str(scale) + ')'] = score_poly
        acc_1nn['(' + str(rot) + ',' + str(scale) + ')'] = acc[0]
        

## MAIN 

In [ ]:
training_feat,tr_lab = load_descriptor_from_matfile('dsc/training_descriptors.mat', 'dsc/training_labels.mat')
tst_features, tst_lab = load_descriptor_from_matfile('dsc/test_descriptors.mat','dsc/test_labels.mat')
tr_lab = tr_lab -1
tst_lab = tst_lab -1

In [ ]:
svm ,score = search_for_best_rbf(training_feat, tr_lab.reshape(-1), tst_features, tst_lab)

In [ ]:
svm ,score = search_for_best_poly(training_feat, tr_lab.reshape(-1), tst_features, tst_lab)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support
acc, prec, rec,clc  = train_model_and_calculate_accuracy(training_feat, tr_lab.reshape(-1), tst_features, tst_lab.reshape(-1))
plot_and_save_knn(acc,prec,rec,'knn_gabor')

In [ ]:
def calculate_and_plot_precision_recall(tst_lab, pred, species, directory, string):
    precision, recall, fbeta, support = precision_recall_fscore_support(tst_lab, pred)
    
    df = pd.DataFrame({"X":species, "precision":precision,"recall":recall,'f1score': fbeta})
    df.plot(x="X", y=["precision", "recall",'f1score'], kind="bar")
    plt.tight_layout()
    plt.savefig(directory +'precision_recall_class4class'+string+'.jpg')
    return np.mean(precision), np.mean(recall), np.mean(fbeta)

In [ ]:

label = np.arange(0,20)

linear_score = svm.score(tst_features,tst_lab)
pred = svm.predict(tst_features)
df = utils.evaluated_prediction(pred, tst_lab, species)
cm = utils.build_confusion_matrix(df, pred, tst_lab,species)
fig=plt.figure(figsize=(30, 15))
utils.plot_confusion_matrix(cm,species,"conf","results_gabor/",normalize=True,title='Confusion matrix')

In [ ]:
calculate_and_plot_precision_recall(tst_lab, pred, species, 'results_gabor/', 'pra.jpg')
os.listdir()